In [43]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
from torchvision import datasets, transforms

import torch.optim as optim
from torch.utils.data import  DataLoader


In [44]:
model = models.resnet18(pretrained=True)

c:\Users\shawk\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\shawk\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [45]:
# Freezing the pretrained layers
for param in model.parameters():
    param.requires_grad = False

In [46]:

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 10)  
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [47]:
# train_transforms = transforms.Compose([
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

# test_transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

In [48]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [49]:
path = "./Data/Original_data/training/training"
val_path = "./Data/Original_data/validation/validation"

train_data = datasets.ImageFolder(root=path,transform=transform)
test_data = datasets.ImageFolder(root=val_path,transform=transform)

In [50]:
train_loader = DataLoader(train_data,batch_size=16,shuffle=True)
val_loader = DataLoader(test_data,batch_size=16,shuffle=True)

In [51]:
learning_rate =0.001
epochs = 5

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=learning_rate,weight_decay=1e-4)


for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images,labels = images.to(device),labels.to(device)

        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

    #validate

    model.eval()
    val_loss = 0.0
    correct = 0
    n_samples = 0
    with torch.no_grad():
        for images,labels in val_loader:
            images,labels = images.to(device),labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs,labels).item()
            _,preds = torch.max(outputs,1)
            correct += (preds == labels).sum().item()
            n_samples += len(labels)
    
    print(f"Validation Loss: {val_loss / len(val_loader)}, Accuracy_val: {(correct / n_samples)*100.0:0.2f}")    



Epoch 1, Loss: 1.3665701068829799
Validation Loss: 0.41202019943910484, Accuracy_val: 94.12
Epoch 2, Loss: 0.40662348918292834
Validation Loss: 0.23648285909610636, Accuracy_val: 93.38
Epoch 3, Loss: 0.2870566482129304
Validation Loss: 0.14874994174084244, Accuracy_val: 95.96
Epoch 4, Loss: 0.19601743328182594
Validation Loss: 0.12133284619845011, Accuracy_val: 97.06
Epoch 5, Loss: 0.14495336847460788
Validation Loss: 0.10568855270085965, Accuracy_val: 97.06
